Plots augmentations for MLCN article

In [1]:
from soma import aims
import numpy as np
import torch

from contrastive.augmentations import RemoveRandomBranchTensor, PartialCutOutTensor_Roll

INFO:numexpr.utils: Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils: NumExpr defaulting to 8 threads.


# Plots data without augmentation

In [2]:
vol = aims.read("/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rlabels/103010_cropped_foldlabel.nii.gz")
skel = aims.read("/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rcrops/103010_cropped_skeleton.nii.gz")

In [3]:
np.unique(vol.np, return_counts=True)

(array([   0,  212, 1001, 1005, 1061, 1079, 1089, 1104, 1132, 1154, 1166,
        1207, 1214, 1226, 1249, 1257, 1298, 1310, 1323, 1327, 1345, 2001,
        2005, 2061, 2062, 2079, 2089, 2104, 2154, 2166, 2181, 2207, 2214,
        2226, 2249, 2257, 2298, 2310, 2323, 2342, 2345, 3259, 3357, 3365,
        3372, 3739, 3754, 3864, 4108, 4117, 4370, 4375, 4376, 4382, 4383,
        4390, 4407, 4600, 4686, 4916, 5349, 5361], dtype=int16),
 array([24890,     1,    21,   107,    15,    54,     4,    39,     2,
           36,    45,     3,     8,    18,    12,    10,    17,     3,
           23,     1,   116,     9,    30,     7,     1,    19,     4,
           18,    17,    13,     1,     8,    12,    16,     9,     6,
            6,     5,     7,     8,    24,     3,    17,     7,    17,
           28,     6,    10,     9,     4,    15,    31,     3,    14,
           20,     4,     1,     1,     1,     2,     1,     1]))

In [4]:
arr_foldlabel = vol.np

In [5]:
np.unique(skel.np, return_counts=True)

(array([  0,  30,  60, 110], dtype=int16), array([24986,   229,   618,     7]))

In [6]:
%rm /tmp/bck_*.bck

In [7]:
random_colors = np.random.random((3,10000))

In [8]:
def colorize(k, max=410., r = random_colors):
    if k//1000 == 1: # surface simple
        # print(k%1000)
        return [r[0,k], r[1,k], r[2,k], 1]
    elif k//1000 == 2: # bottom
        return [r[0,k], 0, 0, 1]
    # elif k//1000 == 3:
    #     return [k%1000 / max, 0, 0, 1]
    # elif k//1000 == 4: # bottom
    #     return [k%1000 / max, 0, 0, 1]
    else:
        return [r[0,k], r[1,k], r[2,k], 1]

In [9]:
def colorize_skel(k):
    if k == 30: # surface simple
        return [1, 0, 0, 1]
    elif k == 60:
        return [0, 0, 1, 1]
    else:
        return [0, 1, 0, 1]

In [10]:
def bucketize_foldlabel(vol, root='bck'):
    c = aims.RawConverter_rc_ptr_Volume_S16_BucketMap_VOID(True)
    bck = aims.BucketMap()
    c.convert(vol, bck)
    for k in bck.keys():
        b2 = aims.BucketMap()
        b2.header().update(bck.header())
        b2[0] = bck[k]
        b2.header()['material'] = {'diffuse': colorize(k)}
        aims.write(b2, f"/tmp/{root}_{k}.bck")   

In [11]:

def bucketize_skeleton(skel, root='bck_skel'):
    c = aims.RawConverter_rc_ptr_Volume_S16_BucketMap_VOID(True)
    bck_skel = aims.BucketMap()
    c.convert(skel, bck_skel)
    
    for k in bck_skel.keys():
        b2 = aims.BucketMap()
        b2.header().update(bck_skel.header())
        b2[0] = bck_skel[k]
        b2.header()['material'] = {'diffuse': colorize_skel(k)}
        aims.write(b2, f"/tmp/{root}_{k}.bck")
    

In [12]:
bucketize_foldlabel(vol)


In [13]:
bucketize_skeleton(skel)

# Plots data with foldlabel augmentation

In [14]:
vol.shape

(17, 40, 38, 1)

In [15]:
branch_clipping = RemoveRandomBranchTensor(torch.from_numpy(vol.np), 40, (17,40,38,1), True, False)

In [16]:
branch_clipped = branch_clipping(torch.from_numpy(skel.np))

In [17]:
np.unique(branch_clipped.numpy(), return_counts=True)

(array([  0.,  30.,  60., 110.], dtype=float32),
 array([25415,     7,   411,     7]))

In [18]:
branch_clipped.shape

torch.Size([17, 40, 38, 1])

In [19]:
arr_skel_clipped = np.array(branch_clipped.numpy().astype(np.int16))

In [20]:
arr_foldlabel_clipped = arr_foldlabel * (arr_skel_clipped != 0)

In [21]:
np.unique(arr_foldlabel)

array([   0,  212, 1001, 1005, 1061, 1079, 1089, 1104, 1132, 1154, 1166,
       1207, 1214, 1226, 1249, 1257, 1298, 1310, 1323, 1327, 1345, 2001,
       2005, 2061, 2062, 2079, 2089, 2104, 2154, 2166, 2181, 2207, 2214,
       2226, 2249, 2257, 2298, 2310, 2323, 2342, 2345, 3259, 3357, 3365,
       3372, 3739, 3754, 3864, 4108, 4117, 4370, 4375, 4376, 4382, 4383,
       4390, 4407, 4600, 4686, 4916, 5349, 5361], dtype=int16)

In [22]:
np.unique(arr_foldlabel_clipped)

array([   0, 1001, 1061, 1079, 1089, 1132, 1154, 1166, 1214, 1226, 1249,
       1257, 1310, 1327, 1345, 3259, 3357, 3365, 3372, 3754, 3864, 4108,
       4117, 4370, 4375, 4376, 4382, 4383, 4390, 4600, 4686, 4916],
      dtype=int16)

In [23]:
foldlabel_clipped = aims.Volume(arr_foldlabel_clipped)

In [24]:
skel_clipped = aims.Volume(np.array(branch_clipped.numpy().astype(np.int16)))

In [25]:
bucketize_skeleton(skel_clipped, 'bck_skel_clipped')

In [26]:
bucketize_foldlabel(foldlabel_clipped, 'bck_foldlabel_clipped')

# Plots data with cut-out augmentation

In [86]:
%rm /tmp/bck_foldlabel_cutout*.bck

In [87]:
cutting_out = PartialCutOutTensor_Roll(True, True, (1,15,30,30), False, None)

In [88]:
cutout = cutting_out(torch.from_numpy(skel.np))

In [89]:
np.unique(cutout.numpy(), return_counts=True)

(array([  0.,  30.,  60., 110.], dtype=float32),
 array([25364,   229,   245,     2]))

In [90]:
arr_cutout = np.array(cutout.numpy().astype(np.int16))

In [91]:
vol_cutout = aims.Volume(arr_cutout)

In [92]:
arr_foldlabel_cutout = arr_foldlabel
arr_foldlabel_cutout = arr_foldlabel_cutout * (arr_cutout != 0)

In [93]:
bucketize_skeleton(vol_cutout, 'bck_skel_cutout')

In [84]:
foldlabel_cutout = aims.Volume(arr_foldlabel_cutout)

In [85]:
bucketize_foldlabel(foldlabel_cutout, 'bck_foldlabel_cutout')